DATA PRE-PROCESSING

In [3]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from scipy.stats import chi2_contingency
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, roc_curve, auc, make_scorer, precision_recall_curve, average_precision_score
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
import pickle
import os
from mlxtend.feature_selection import SequentialFeatureSelector

In [4]:
## Load the client and invoice data
data_client = pd.read_csv(os.path.join('..', 'data_files_and_model', 'client.csv'))
data_invoice = pd.read_csv(os.path.join('..', 'data_files_and_model', 'invoice.csv'))

In [5]:
## Identify missing data
print(data_client.isnull().sum())
print(data_invoice.isnull().sum())

region    0
date      0
dis       0
id        0
catg      0
target    0
dtype: int64
id                      0
date                    0
tarif_type              0
counter_statue          0
reading_remarque        0
consommation_level_4    0
months_number           0
counter_type            0
counter_coefficient     0
consommation_level_1    0
consommation_level_2    0
consommation_level_3    0
dtype: int64


It can be observed that there are no missing data!

In [6]:
## Convert date columns to datetime format
data_client['date'] = pd.to_datetime(data_client['date'], errors='coerce')
data_invoice['date'] = pd.to_datetime(data_invoice['date'], errors='coerce')

## Check if any date conversions failed (i.e., resulted in NaT)
print(data_client[data_client['date'].isna()])
print(data_invoice[data_invoice['date'].isna()])

Empty DataFrame
Columns: [region, date, dis, id, catg, target]
Index: []
Empty DataFrame
Columns: [id, date, tarif_type, counter_statue, reading_remarque, consommation_level_4, months_number, counter_type, counter_coefficient, consommation_level_1, consommation_level_2, consommation_level_3]
Index: []


/var/folders/t2/_hypj1cj7gj6b3j_nd6zhsgr0000gn/T/ipykernel_69390/2964087332.py:2: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data_client['date'] = pd.to_datetime(data_client['date'], errors='coerce')
/var/folders/t2/_hypj1cj7gj6b3j_nd6zhsgr0000gn/T/ipykernel_69390/2964087332.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  data_invoice['date'] = pd.to_datetime(data_invoice['date'], errors='coerce')


In [7]:
# Convert date column to datetime
data_client['date'] = pd.to_datetime(data_client['date'])

# Extract year, month, and tenure
data_client['year_joined'] = data_client['date'].dt.year
data_client['month_joined'] = data_client['date'].dt.month
data_client['tenure'] = (pd.to_datetime('today') - data_client['date']).dt.days

# Convert to datetime and extract features
data_invoice['date'] = pd.to_datetime(data_invoice['date'])
data_invoice['year'] = data_invoice['date'].dt.year
data_invoice['month'] = data_invoice['date'].dt.month
data_invoice['days_since_last_invoice'] = (pd.to_datetime('today') - data_invoice['date']).dt.days

print(data_client.head())
print(data_invoice.head())

   region       date  dis    id  catg  target  year_joined  month_joined  \
0     101 1994-12-31   60     0    11       0         1994            12   
1     107 2002-05-29   69     1    11       0         2002             5   
2     301 1986-03-13   62    10    11       0         1986             3   
3     105 1996-07-11   69   100    11       0         1996             7   
4     303 2014-10-14   62  1000    11       0         2014            10   

   tenure  
0   10900  
1    8194  
2   14115  
3   10342  
4    3673  
   id       date  tarif_type  counter_statue  reading_remarque  \
0   0 2014-03-24          11               0                 8   
1   0 2013-03-29          11               0                 6   
2   0 2015-03-23          11               0                 8   
3   0 2015-07-13          11               0                 8   
4   0 2016-11-17          11               0                 9   

   consommation_level_4  months_number counter_type  counter_coefficient  

In [8]:
data_client = data_client.drop(columns=['date', 'year_joined', 'month_joined'])
data_invoice = data_invoice.drop(columns=['date'])

In [9]:
## Merging data sets based on ' id' variable using inner join
merged_df = pd.merge(data_invoice, data_client, on='id', how='inner')

## Preview the merged dataframe
print(merged_df.head())

   id  tarif_type  counter_statue  reading_remarque  consommation_level_4  \
0   0          11               0                 8                     0   
1   0          11               0                 6                     0   
2   0          11               0                 8                     0   
3   0          11               0                 8                     0   
4   0          11               0                 9                     0   

   months_number counter_type  counter_coefficient  consommation_level_1  \
0              4         ELEC                    1                    82   
1              4         ELEC                    1                  1200   
2              4         ELEC                    1                   123   
3              4         ELEC                    1                   102   
4             12         ELEC                    1                   572   

   consommation_level_2  consommation_level_3  year  month  \
0                 

In [10]:
## Converting counter_statue into a categorical variable
merged_df['counter_statue'] = merged_df['counter_statue'].astype('category').cat.codes

In [11]:
## Checking for validity of data and variables
for column in merged_df.columns:
    print("COLUMN NAME:", column, "\nunique values:\n", merged_df[column].unique(), "\n\n")

COLUMN NAME: id 
unique values:
 [     0      1     10 ... 128436 128437 128438] 


COLUMN NAME: tarif_type 
unique values:
 [11 40 15 10 45 29 13 12 14  9 30 21] 


COLUMN NAME: counter_statue 
unique values:
 [0 1 5 4 3 2] 


COLUMN NAME: reading_remarque 
unique values:
 [8 6 9 7] 


COLUMN NAME: consommation_level_4 
unique values:
 [   0  382  132 ... 1978 3591 2312] 


COLUMN NAME: months_number 
unique values:
 [     4     12      8      2      6     16      1     10      7     14
     15      5     18      3     24     20     42     32     22      9
     11     28  25778  17815   9971   3311     30    495     40     52
  83125  30144  99104   7467  27928  54894  55126     56     25     36
     26     44     13     82     35     48     62   5638   4252   5479
     60     17     50     34     21 152508 166946 159443 231602 228522
 231136 230328 229399 229113 229026 228836 227889 227582 227015 226612
 226284 228242 229890 226001  17960  16734  16676  16218  17633  17902
  16883  1

Upon inspection of the different columns, it can be observed that 'months_number' has values greater than 12, hence these entries must be removed

In [12]:
## Dropping of rows with months_number > 12
merged_df = merged_df[(merged_df['months_number'] < 13) & (merged_df['months_number'] > 0)]
print(merged_df['months_number'].unique())

[ 4 12  8  2  6  1 10  7  5  3  9 11]


In [13]:
from sklearn.model_selection import train_test_split

## Define features and target
xval = merged_df.drop(columns=['target', 'counter_type'])
yval = merged_df['target']

## Split the data train and test sets
xtrain, xtest, ytrain, ytest = train_test_split(xval, yval, test_size=0.3, random_state=0)

print('xtrain shape:', xtrain.shape)
print('xtest shape:', xtest.shape)
print('ytrain shape:', ytrain.shape)
print('ytest shape:', ytest.shape)
print('ytrain values:', ytrain.value_counts())
print('ytest values:', ytest.value_counts())

xtrain shape: (347463, 17)
xtest shape: (148914, 17)
ytrain shape: (347463,)
ytest shape: (148914,)
ytrain values: target
0    325550
1     21913
Name: count, dtype: int64
ytest values: target
0    139742
1      9172
Name: count, dtype: int64


In [14]:
print(xval.columns)
print(xval.dtypes)

Index(['id', 'tarif_type', 'counter_statue', 'reading_remarque',
       'consommation_level_4', 'months_number', 'counter_coefficient',
       'consommation_level_1', 'consommation_level_2', 'consommation_level_3',
       'year', 'month', 'days_since_last_invoice', 'region', 'dis', 'catg',
       'tenure'],
      dtype='object')
id                         int64
tarif_type                 int64
counter_statue              int8
reading_remarque           int64
consommation_level_4       int64
months_number              int64
counter_coefficient        int64
consommation_level_1       int64
consommation_level_2       int64
consommation_level_3       int64
year                       int32
month                      int32
days_since_last_invoice    int64
region                     int64
dis                        int64
catg                       int64
tenure                     int64
dtype: object


In [15]:
## Standardising the train and test data

from sklearn.preprocessing import StandardScaler
std_x = StandardScaler()
xtrain = pd.DataFrame(std_x.fit_transform(xtrain), columns = xval.columns)
xtest = pd.DataFrame(std_x.transform(xtest), columns = xval.columns)

xtrain = xtrain.reset_index(drop=True)
ytrain = ytrain.reset_index(drop=True)
xtest = xtest.reset_index(drop=True)
ytest = ytest.reset_index(drop=True)


print('xtrain shape:', xtrain.shape)
print('xtest shape:', xtest.shape)

xtrain shape: (347463, 17)
xtest shape: (148914, 17)


In [16]:
xtrain[:10]

,id,tarif_type,counter_statue,reading_remarque,consommation_level_4,months_number,counter_coefficient,consommation_level_1,consommation_level_2,consommation_level_3,year,month,days_since_last_invoice,region,dis,catg,tenure
0,0.001822,2.142655,-0.126643,1.116655,-0.050074,-0.190056,-0.003118,-0.062907,-0.07541,-0.182923,0.937450,-1.566466,-0.845195,-0.956749,1.617151,-0.096891,-0.784811
1,0.568205,-0.458563,-0.126643,0.389850,-0.050074,-0.190056,-0.003118,0.316387,-0.07541,-0.182923,0.003044,-1.566466,0.109513,-0.995001,-1.040364,-0.096891,-0.498668
2,0.081689,-0.458563,-0.126643,1.116655,-0.050074,-0.190056,-0.003118,-0.191226,-0.07541,-0.182923,0.937450,-1.566466,-0.836219,-0.975875,1.617151,-0.096891,0.924262
3,0.411359,-0.458563,-0.126643,1.116655,-0.050074,-0.190056,-0.003118,-0.079890,-0.07541,-0.182923,1.404653,0.470279,-1.444052,1.586981,-0.449805,-0.096891,-0.855593
4,0.036293,-0.458563,-0.126643,-1.063761,-0.050074,-0.190056,-0.003118,-0.791303,-0.07541,-0.182923,-1.398565,0.179315,1.392502,0.965393,-0.154525,-0.096891,1.306036
5,0.447463,-0.458563,-0.126643,-1.063761,-0.050074,-1.406620,-0.003118,-0.632792,-0.07541,-0.182923,0.703849,-0.984539,-0.644508,-0.966312,1.617151,-0.096891,0.446856
6,0.727342,-0.458563,-0.126643,-1.063761,-0.050074,-0.190056,-0.003118,-0.819609,-0.07541,-0.182923,0.236646,0.470279,-0.263650,1.032333,-0.154525,-0.096891,-0.459763
7,-0.050889,2.142655,-0.126643,0.389850,-0.050074,-0.190056,-0.003118,-0.796964,-0.07541,-0.182923,0.703849,1.052206,-0.768895,0.009103,1.617151,-0.096891,-0.889729
8,-2.870525,-0.458563,-0.126643,0.389850,-0.050074,-0.190056,-0.003118,-0.795077,-0.07541,-0.182923,-0.230557,-0.111648,0.239671,-0.975875,1.617151,-0.096891,-0.848816
9,-2.869175,-0.458563,-0.126643,1.116655,-0.050074,-0.190056,-0.003118,-0.036489,-0.07541,-0.182923,1.404653,-0.984539,-1.349158,0.994082,-0.449805,-0.096891,-0.756698


In [17]:
ytrain[:10]

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
8    0
9    0
Name: target, dtype: int64

BUILDING LOGISTIC REGRESSION MODEL

Using train and test sets for training and prediction respectively


In [18]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Initialize the logistic regression model
log_reg = LogisticRegression(random_state=0)

# Fit the model to the training data
log_reg.fit(xtrain, ytrain)

LogisticRegression(random_state=0)

In [19]:
y_pred = log_reg.predict(xtest)

# Print evaluation metrics
print("Accuracy:", accuracy_score(ytest, y_pred))
print("Confusion Matrix:\n", confusion_matrix(ytest, y_pred))
print("Classification Report:\n", classification_report(ytest, y_pred))

Accuracy: 0.9383805417892206
Confusion Matrix:
 [[139737      5]
 [  9171      1]]
Classification Report:
               precision    recall  f1-score   support

           0       0.94      1.00      0.97    139742
           1       0.17      0.00      0.00      9172

    accuracy                           0.94    148914
   macro avg       0.55      0.50      0.48    148914
weighted avg       0.89      0.94      0.91    148914



In [20]:
import statsmodels.api as sm

def backward_elimination(x, y, significance_level = 0.05):
    features = x.columns.tolist()
    while(len(features)>0):
        features_with_constant = sm.add_constant(x[features])
        p_values = sm.OLS(y, features_with_constant).fit().pvalues[1:]
        max_p_value = p_values.max()
        if(max_p_value >= significance_level):
            excluded_feature = p_values.idxmax()
            features.remove(excluded_feature)
        else:
            break 
    return features

In [21]:
new_features = backward_elimination(xtrain, ytrain)
print(new_features)

['id', 'tarif_type', 'counter_statue', 'reading_remarque', 'consommation_level_4', 'months_number', 'consommation_level_1', 'consommation_level_3', 'year', 'month', 'days_since_last_invoice', 'region', 'dis', 'catg', 'tenure']


In [22]:
xtrain_selected = xtrain[new_features]
xtest_selected = xtest[new_features]

xtrain_selected[:10]

,id,tarif_type,counter_statue,reading_remarque,consommation_level_4,months_number,consommation_level_1,consommation_level_3,year,month,days_since_last_invoice,region,dis,catg,tenure
0,0.001822,2.142655,-0.126643,1.116655,-0.050074,-0.190056,-0.062907,-0.182923,0.937450,-1.566466,-0.845195,-0.956749,1.617151,-0.096891,-0.784811
1,0.568205,-0.458563,-0.126643,0.389850,-0.050074,-0.190056,0.316387,-0.182923,0.003044,-1.566466,0.109513,-0.995001,-1.040364,-0.096891,-0.498668
2,0.081689,-0.458563,-0.126643,1.116655,-0.050074,-0.190056,-0.191226,-0.182923,0.937450,-1.566466,-0.836219,-0.975875,1.617151,-0.096891,0.924262
3,0.411359,-0.458563,-0.126643,1.116655,-0.050074,-0.190056,-0.079890,-0.182923,1.404653,0.470279,-1.444052,1.586981,-0.449805,-0.096891,-0.855593
4,0.036293,-0.458563,-0.126643,-1.063761,-0.050074,-0.190056,-0.791303,-0.182923,-1.398565,0.179315,1.392502,0.965393,-0.154525,-0.096891,1.306036
5,0.447463,-0.458563,-0.126643,-1.063761,-0.050074,-1.406620,-0.632792,-0.182923,0.703849,-0.984539,-0.644508,-0.966312,1.617151,-0.096891,0.446856
6,0.727342,-0.458563,-0.126643,-1.063761,-0.050074,-0.190056,-0.819609,-0.182923,0.236646,0.470279,-0.263650,1.032333,-0.154525,-0.096891,-0.459763
7,-0.050889,2.142655,-0.126643,0.389850,-0.050074,-0.190056,-0.796964,-0.182923,0.703849,1.052206,-0.768895,0.009103,1.617151,-0.096891,-0.889729
8,-2.870525,-0.458563,-0.126643,0.389850,-0.050074,-0.190056,-0.795077,-0.182923,-0.230557,-0.111648,0.239671,-0.975875,1.617151,-0.096891,-0.848816
9,-2.869175,-0.458563,-0.126643,1.116655,-0.050074,-0.190056,-0.036489,-0.182923,1.404653,-0.984539,-1.349158,0.994082,-0.449805,-0.096891,-0.756698


In [23]:
xtrain = xtrain.reset_index(drop=True)
ytrain = ytrain.reset_index(drop=True)
xtest = xtest.reset_index(drop=True)
ytest = ytest.reset_index(drop=True)

In [24]:
# Initialize the logistic regression model
log_reg_selected = LogisticRegression(random_state=1)

# Fit the model to the training data
log_reg_selected.fit(xtrain_selected, ytrain)
y_pred_selected = log_reg_selected.predict(xtest_selected)

# Print evaluation metrics
print("Accuracy:", accuracy_score(ytest, y_pred_selected))
print("Confusion Matrix:\n", confusion_matrix(ytest, y_pred_selected))
print("Classification Report:\n", classification_report(ytest, y_pred_selected))

Accuracy: 0.9383805417892206
Confusion Matrix:
 [[139737      5]
 [  9171      1]]
Classification Report:
               precision    recall  f1-score   support

           0       0.94      1.00      0.97    139742
           1       0.17      0.00      0.00      9172

    accuracy                           0.94    148914
   macro avg       0.55      0.50      0.48    148914
weighted avg       0.89      0.94      0.91    148914



In [25]:
## Print coefficients with feature names
for feature, coef in zip(new_features, log_reg_selected.coef_[0]):
    print(f"{feature}: {coef:.4f}")

id: -0.0540
tarif_type: 0.0151
counter_statue: 0.0279
reading_remarque: 0.0526
consommation_level_4: -0.0341
months_number: -0.0257
consommation_level_1: 0.0565
consommation_level_3: 0.1129
year: 0.0390
month: -0.0108
days_since_last_invoice: -0.0373
region: 0.1359
dis: 0.3004
catg: 0.1359
tenure: 0.2616
